In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # elimina los errres de que van a cambiar

# importamos las librerías que necesitamos
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

from word2number import w2n

In [2]:
df = pd.read_csv('DisneyMoviesDataset.csv', index_col=0)

In [3]:
df.sample(10)

,title,Production company,Release date,Running time,Country,Language,Running time (int),Budget (float),Box office (float),Release date (datetime),imdb,metascore,rotten_tomatoes,Directed by,Produced by,Written by,Based on,Starring,Music by,Distributed by,Budget,Box office,Story by,Narrated by,Cinematography,Edited by,Screenplay by,Production companies,Adaptation by,Traditional,Simplified
76,That Darn Cat!,Walt Disney Productions,"['December 2, 1965']",116 minutes,United States,English,116.0,NaN,2.806822e+07,1965-12-02,6.8,71.0,94%,Robert Stevenson,"['Walt Disney', 'Bill Walsh']","['Gordon Gordon', 'Mildred Gordon', 'Bill Walsh']",NaN,"['Hayley Mills', 'Dean Jones', 'Dorothy Provin...",Robert F. Brunner,Buena Vista Distribution,NaN,"$28,068,222",NaN,NaN,Edward Colman,Cotton Warburton,NaN,NaN,NaN,NaN,NaN
413,Frozen II,"['Walt Disney Pictures', 'Walt Disney Animatio...","['November 7, 2019 ( Dolby Theatre )', 'Novemb...",103 minutes,United States,English,103.0,150000000.0,1.450000e+09,2019-11-07,6.9,64.0,77%,"['Chris Buck', 'Jennifer Lee']",Peter Del Vecho,NaN,NaN,"['Kristen Bell', 'Idina Menzel', 'Josh Gad', '...","['Christophe Beck', 'Robert Lopez', 'Kristen A...","['Walt Disney Studios', 'Motion Pictures']",$150 million,$1.450 billion,"['Chris Buck', 'Jennifer Lee', 'Marc E. Smith'...",NaN,"['Tracy Scott Beattie (layout)', 'Mohit Kallia...",Jeff Draheim,['Jennifer Lee'],NaN,NaN,NaN,NaN
113,Herbie Rides Again,Walt Disney Productions,"['June 6, 1974']",88 minutes,United States,English,88.0,NaN,3.822900e+07,1974-06-06,5.7,49.0,80%,Robert Stevenson,Bill Walsh,NaN,"['Characters', 'by Gordon Buford']","['Helen Hayes', 'Ken Berry', 'Stefanie Powers'...",George Bruns,Buena Vista Distribution,NaN,"['$38,229,000 (US/Canada gross)', '$30.8 milli...",Gordon Buford,NaN,Frank V. Phillips,Cotton Warburton,Bill Walsh,NaN,NaN,NaN,NaN
47,Jungle Cat,Walt Disney Productions,"['August 10, 1960']",69 minutes,United States,English,69.0,NaN,2.300000e+06,1960-08-10,7.4,NaN,NaN,James Algar,Ben Sharpsteen,James Algar,NaN,NaN,Oliver Wallace,Buena Vista Distribution,NaN,$2.3 million (US and Canadian rentals),NaN,Winston Hibler,"['Lloyd Beebe', 'James R. Simon', 'Hugh A. Wil...",Norman R. Palmer,NaN,NaN,NaN,NaN,NaN
128,Escape from the Dark,Walt Disney Productions,"['May 26, 1976', '(UK)', 'March 11, 1977', '(U...",104 minutes,"['United Kingdom', 'United States']",English,104.0,NaN,NaN,1976-05-26,NaN,NaN,NaN,Charles Jarrott,Ron Miller,Rosemary Anne Sisson,NaN,"['Alastair Sim', 'Peter Barkworth', 'Maurice C...",Ron Goodwin,Buena Vista Distribution,NaN,NaN,"['Burt Kennedy', 'Rosemary Anne Sisson']",NaN,Paul Beeson,Richard Marden,NaN,NaN,NaN,NaN,NaN
398,Coco,"['Walt Disney Pictures', 'Pixar Animation Stud...","['October 20, 2017 ( Morelia )', 'November 22,...",105 minutes,United States,English,105.0,175000000.0,8.071000e+08,2017-10-20,8.4,81.0,97%,Lee Unkrich,Darla K. Anderson,NaN,NaN,"['Anthony Gonzalez', 'Gael García Bernal', 'Be...",Michael Giacchino,"['Walt Disney Studios', 'Motion Pictures']",$175 million,$807.1 million,"['Lee Unkrich', 'Jason Katz', 'Matthew Aldrich...",NaN,"['Matt Aspbury', 'Danielle Feinberg']",Steve Bloom,"['Adrian Molina', 'Matthew Aldrich']",NaN,NaN,NaN,NaN
33,Westward Ho the Wagons!,Walt Disney Productions,"['December 20, 1956']",90 min.,United States,English,90.0,NaN,2.750000e+06,1956-12-20,5.8,NaN,NaN,William Beaudine,Bill Walsh,NaN,a novel by Mary Jane Carr,"['Fess Parker', 'Kathleen Crowley', 'Jeff York...",George Bruns,Buena Vista Distribution,NaN,$2.75 million (US),NaN,NaN,"Charles P. Boyle , A.S.C.","Cotton Warburton , A.C.E.",Tom Blackburn,NaN,NaN,NaN,NaN
317,Race to Witch Mountain,"['Walt Disney Pictures', 'Gunn Films']","['March 11, 2009 (Egypt)', 'March 13, 2009 (Un...",98 minutes,United States,English,98.0,50000000.0,1.064000e+08,2009-03-11,5.7,52.0,43%,Andy Fickman,Andrew Gunn,NaN,"['Escape to Witch Mountain', 'by', 'Alexander ...","['Dwayne Johnson', 'AnnaSophia Robb', 'Carla G...",Trevor Rabin,"['Walt Disney Studios',

In [4]:
df.shape

(432, 31)

In [5]:
df.tail()

,title,Production company,Release date,Running time,Country,Language,Running time (int),Budget (float),Box office (float),Release date (datetime),imdb,metascore,rotten_tomatoes,Directed by,Produced by,Written by,Based on,Starring,Music by,Distributed by,Budget,Box office,Story by,Narrated by,Cinematography,Edited by,Screenplay by,Production companies,Adaptation by,Traditional,Simplified
427,Soul,"['Walt Disney Pictures', 'Pixar Animation Stud...","['October 15, 2020 ( Rome Film Festival )', 'N...",NaN,United States,English,NaN,150000000.0,NaN,2020-10-15,7.0,53.0,45%,Pete Docter,Dana Murray,NaN,NaN,"['Jamie Foxx', 'Tina Fey', 'Questlove', 'Phyli...","['Trent Reznor', 'Atticus Ross']","['Walt Disney Studios', 'Motion Pictures']",$150 million+,NaN,Pete Docter,NaN,NaN,NaN,"['Pete Docter', 'Mike Jones', 'Kemp Powers']",NaN,NaN,NaN,NaN
428,Raya and the Last Dragon,"['Walt Disney Pictures', 'Walt Disney Animatio...","['March 12, 2021']",NaN,United States,English,NaN,NaN,NaN,2021-03-12,NaN,NaN,NaN,"['Don Hall', 'Carlos López Estrada']","['Osnat Shurer', 'Peter Del Vecho']","['Qui Nguyen', 'Adele Lim']",NaN,"['Kelly Marie Tran', 'Awkwafina']",NaN,"['Walt Disney Studios', 'Motion Pictures']",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,Cruella,"['Walt Disney Pictures', 'Gunn Films', 'Marc P...","['May 28, 2021 (United States)']",NaN,United States,English,NaN,NaN,NaN,2021-05-28,NaN,NaN,NaN,Craig Gillespie,"['Kristin Burr', 'Andrew Gunn', 'Marc Platt']",NaN,"[""Disney's One Hundred and One Dalmatians by B...","['Emma Stone', 'Emma Thompson', 'Paul Walter H...",NaN,Walt Disney Studios Motion Pictures,NaN,NaN,NaN,NaN,Nicolas Karakatsanis,Tatiana S. Riegel,"['Aline Brosh McKenna', 'Jez Butterworth', 'Da...",NaN,NaN,NaN,NaN
430,Jungle Cruise,"['Walt Disney Pictures', 'Davis Entertainment'...","['July 30, 2021 (United States)']",NaN,United States,English,NaN,NaN,NaN,2021-07-30,NaN,NaN,NaN,Jaume Collet-Serra,"['John Davis', 'John Fox', 'Beau Flynn', 'Dway...",NaN,"['Jungle Cruise', 'by', 'Walt Disney']","['Dwayne Johnson', 'Emily Blunt', 'Édgar Ramír...","['James Newton Howard', 'Metallica']",Walt Disney Studios Motion Pictures,NaN,NaN,"['Glenn Ficarra', 'John Requa']",NaN,Flavio Labiano,Joel Negron,"['Michael Green', 'Glenn Ficarra', 'John Requa']",NaN,NaN,NaN,NaN
431,The Beatles: Get Back,NaN,['27 August 2021'],NaN,"['United Kingdom', 'New Zealand', 'United Stat...",English,NaN,NaN,NaN,2021-08-27,8.0,NaN,NaN,Peter Jackson,"['Peter Jackson', 'Clare Olssen', 'Jonathan Cl...",NaN,NaN,The Beatles,The Beatles,"['Walt Disney Studios', 'Motion Pictures']",NaN,NaN,NaN,NaN,NaN,Jabez Olssen,NaN,"['Walt Disney Pictures', 'Apple Corps', 'WingN...",NaN,NaN,NaN


In [6]:
#Creamos una función para entender el data frame


def entender_datos(df):
    
    # Encabezado
    print("INFO SOBRE NUESTROS DATOS")
    print("============================================================================")
    
    # Información general
    print(f"El número de filas es: {df.shape[0]}")
    print(f"El número de columnas es: {df.shape[1]}")
    print("============================================================================")
    
    # Descripción del DataFrame
    print("DESCRIPCIÓN ESTADÍSTICA")
    print(df.describe().T)
    print("============================================================================")
    
    # Columnas del DataFrame
    print("COLUMNAS")
    print(df.columns)
    print("============================================================================")
    
    # Filas duplicadas
    print(f"Número de filas duplicadas: {df.duplicated().sum()}")
    print("============================================================================")
    
    # Valores nulos totales
    print(f"Total de valores nulos: {df.isna().sum().sum()}")
    print("============================================================================")
    
    # Valores nulos por columna
    print("VALORES NULOS POR COLUMNA")
    print(df.isna().sum())
    print("============================================================================")

    #Porcentaje de valores nulos por columna
    print("PORCENTAJE NULOS POR COLUMNA")
    print(round(df.isna().sum() / df.shape[0] * 100,2))
    print("============================================================================")

    # Tipos de datos
    print("TIPOS DE DATOS")
    print(df.dtypes.value_counts())
    print("============================================================================")
    
    # Información del DataFrame
    print("INFORMACIÓN DETALLADA")
    df.info()
    print("============================================================================")



In [7]:
df['Country'].unique()

array(['United States', "['United States', 'United Kingdom']",
       "['United Kingdom', 'United States']", 'United States, Mexico',
       "['United States', 'Canada']", nan,
       "['United States', 'Australia']", "['United States', 'Norway']",
       'Australia', "['Norway', 'Sweden', 'United States']",
       "['United States', 'Austria']", "['United States', 'France']",
       "['United States', 'United Kingdom', 'Germany']",
       "['United States', 'United Kingdom', 'France']",
       "['Canada', 'Malaysia', 'United States']",
       "['France', 'United Kingdom', 'Germany', 'United States']",
       "['Germany', 'Austria', 'Italy', 'Spain', 'United Kingdom']",
       'Russia', 'France', 'India', 'Germany',
       "['United States', 'France', 'United Kingdom']",
       "['France', 'United States']",
       "['Australia', 'United Kingdom', 'United States']",
       "['Spain', 'Italy']", "['United States']",
       "['United States', 'China', 'France']",
       "['United Kingdom

In [8]:
entender_datos(df)

INFO SOBRE NUESTROS DATOS
El número de filas es: 432
El número de columnas es: 31
DESCRIPCIÓN ESTADÍSTICA
                    count          mean           std    min         25%  \
Running time (int)  422.0  9.730569e+01  1.895949e+01   40.0        86.0   
Budget (float)      273.0  6.358861e+07  7.163732e+07  150.0  10000000.0   
Box office (float)  355.0  1.674014e+08  2.749517e+08    7.7   9850000.0   
imdb                416.0  6.545433e+00  9.609374e-01    1.5         6.0   
metascore           283.0  6.148763e+01  1.656266e+01   18.0        49.0   

                           50%           75%           max  
Running time (int)        96.0  1.067500e+02  1.680000e+02  
Budget (float)      30000000.0  1.000000e+08  4.106000e+08  
Box office (float)  42900000.0  1.865500e+08  1.657000e+09  
imdb                       6.6  7.200000e+00  8.700000e+00  
metascore                 61.0  7.300000e+01  9.900000e+01  
COLUMNAS
Index(['title', 'Production company', 'Release date', 'Running

 0   title                    432 non-null    object  OK  ***SI***

 1   Production company       392 non-null    object  Lista (ver como hacer) (ver como hacer) Hacer explode. USA --- Nulos ***SI***

 2   Release date             431 non-null    object  Tenemos lista hay otra columna con este dato Podemos hacer una nueva columna con el lugar de estreno dividido entre parentesis ***SI*** LUGAR DE ESTENO ()

 3   Running time             422 non-null    object  ***ELIMINAR COLUMNA***

 4   Country                  428 non-null    object  Lista (ver como hacer) Hacer explode ***SI***

 5   Language                 430 non-null    object  Lista (ver como hacer) Hacer explode ***SI***

 6   Running time (int)       422 non-null    float64 OK coger media para nulos ***SI***

 7   Budget (float)           273 non-null    float64 Ok gestionar nulos ***SI***

 8   Box office (float)       355 non-null    float64 ok gestionar nulos ***SI*** (GANADO EN TAQUILLA)

 9   Release date (datetime)  428 non-null    object  ok los nulos como una categoria nueva ***SI*** 

 10  imdb                     416 non-null    float64 ok  media mediana para nulos ***SI*** 

 11  metascore                283 non-null    float64  ***ELIMINAR COLUMNA***

 12  rotten_tomatoes          361 non-null    object  ***ELIMINAR COLUMNA***

 13  Directed by              431 non-null    object  tiene listas, quededarnos con el primero pero quitar las categorias que no vamos a usar ***SI*** 

 14  Produced by              422 non-null    object nos quedamos con el primero ***SI*** 

 15  Written by               203 non-null    object idem directed by ***ELIMINAR COLUMNA***

 16  Based on                 211 non-null    object lista convertir string, limpieza de caracteres ***SI*** 

 17  Starring                 398 non-null    object explode es una lista, nos quedamos con todo ***ELIMINAR COLUMNA***

 18  Music by                 422 non-null    object es una lista, nos quedamos con el primero ***ELIMINAR COLUMNA***

 19  Distributed by           429 non-null    object es una lista, nos quedamos con el primero ***SI***

 20  Budget                   276 non-null    object ***ELIMINAR COLUMNA***

 21  Box office               359 non-null    object ***ELIMINAR COLUMNA***

 22  Story by                 139 non-null    object es una lista, nos quedamos con el primero ***ELIMINAR COLUMNA***

 23  Narrated by              55 non-null     object ***ELIMINAR COLUMNA***

 24  Cinematography           342 non-null    object es una lista, nos quedamos con el primero, y nombres ***ELIMINAR COLUMNA***

 25  Edited by                395 non-null    object es una lista, nos quedamos con el primero, y nombres ***ELIMINAR COLUMNA***

 26  Screenplay by            191 non-null    object tiene listas, quededarnos con el primero pero quitar las categorias que no vamos a usar ***ELIMINAR COLUMNA***

 27  Production companies     39 non-null     object ***ELIMINAR COLUMNA***

 28  Adaptation by            1 non-null      object ***ELIMINAR COLUMNA***

 29  Traditional              1 non-null      object ***ELIMINAR COLUMNA***

 30  Simplified               1 non-null      object ***ELIMINAR COLUMNA***

In [9]:
df['Distributed by'].unique()

array([nan, 'RKO Radio Pictures',
       "['Walt Disney Productions', 'RKO Radio Pictures']",
       'United Artists', 'RKO Radio Pictures, Inc.',
       'Buena Vista Distribution',
       'Buena Vista Film Distribution Co., Inc.',
       'Buena Vista Film Distribution',
       'Buena Vista Pictures Distribution',
       'Buena Vista Distribution Co., Inc.',
       "['Buena Vista Distribution', '(US)', 'British Empire Films', '(Australia)']",
       "['Paramount Pictures (North America)', '(North America)', 'Buena Vista International Distribution (International)', '(International)']",
       "['Paramount Pictures', 'Buena Vista International Distribution']",
       "['PSO Productions (outside North America)', 'Buena Vista Distribution']",
       'Buena Vista Pictures',
       "['Buena Vista Pictures', '(North America and other select territories)', 'MDP Worldwide', '(all other territories)']",
       "['Buena Vista Pictures', '(United States)', 'Trimark Pictures (International)']",
   

In [10]:
df.head()


,title,Production company,Release date,Running time,Country,Language,Running time (int),Budget (float),Box office (float),Release date (datetime),imdb,metascore,rotten_tomatoes,Directed by,Produced by,Written by,Based on,Starring,Music by,Distributed by,Budget,Box office,Story by,Narrated by,Cinematography,Edited by,Screenplay by,Production companies,Adaptation by,Traditional,Simplified
0,Academy Award Review of,Walt Disney Productions,"['May 19, 1937']",41 minutes (74 minutes 1966 release),United States,English,41.0,NaN,NaN,1937-05-19,7.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Snow White and the Seven Dwarfs,Walt Disney Productions,"['December 21, 1937 ( Carthay Circle Theatre ,...",83 minutes,United States,English,83.0,1490000.0,418000000.0,1937-12-21,7.6,95.0,NaN,"['David Hand (supervising)', 'William Cottrell...",Walt Disney,"['Ted Sears', 'Richard Creedon', 'Otto England...","['Snow White', 'by The', 'Brothers Grimm']","['Adriana Caselotti', 'Lucille La Verne', 'Har...","['Frank Churchill', 'Paul Smith', 'Leigh Harli...",RKO Radio Pictures,$1.49 million,$418 million,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pinocchio,Walt Disney Productions,"['February 7, 1940 ( Center Theatre )', 'Febru...",88 minutes,United States,English,88.0,2600000.0,164000000.0,1940-02-07,7.4,99.0,100%,"['Ben Sharpsteen', 'Hamilton Luske', 'Bill Rob...",Walt Disney,NaN,"['The Adventures of Pinocchio', 'by', 'Carlo C...","['Cliff Edwards', 'Dickie Jones', 'Christian R...","['Leigh Harline', 'Paul J. Smith']",RKO Radio Pictures,$2.6 million,$164 million,"['Ted Sears', 'Otto Englander', 'Webb Smith', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fantasia,Walt Disney Productions,"['November 13, 1940']",126 minutes,United States,English,126.0,2280000.0,83300000.0,1940-11-13,7.8,96.0,95%,"['Samuel Armstrong', 'James Algar', 'Bill Robe...","['Walt Disney', 'Ben Sharpsteen']",NaN,NaN,"['Leopold Stokowski', 'Deems Taylor']",See program,"['Walt Disney Productions', 'RKO Radio Pictures']",$2.28 million,$76.4–$83.3 million,"['Joe Grant', 'Dick Huemer']",Deems Taylor,James Wong Howe,NaN,NaN,NaN,NaN,NaN,NaN
4,The Reluctant Dragon,Walt Disney Productions,"['June 20, 1941']",74 minutes,United States,English,74.0,600000.0,960000.0,1941-06-20,6.9,NaN,67%,"['Alfred Werker', '(live action)', 'Hamilton L...",Walt Disney,"['Live-action:', 'Ted Sears', 'Al Perkins', 'L...",NaN,"['Robert Benchley', 'Frances Gifford', 'Buddy ...","['Frank Churchill', 'Larry Morey']",RKO Radio Pictures,"$600,000","$960,000 (worldwide rentals)",NaN,NaN,Bert Giennon,Paul Weatherwax,NaN,NaN,NaN,NaN,NaN


In [11]:
df2_cat = df.select_dtypes(include = "object")


# creamos una variable con los nombres de las columnas del DataFrame de las variables categóricas utilizando el método '.columns'
columnas_cat = df2_cat.columns
print(f"Las columnas del DataFrame de variables categóricas son {columnas_cat}")

# empezamos a iterar por cada una de las columnas para sacar sus valores únicos y sus frecuencias
for columna in columnas_cat:
    print(f" \n----------- ESTAMOS ANALIZANDO LA COLUMNA: '{columna.upper()}' -----------\n")
    print(f"Sus valores únicos son: {df2_cat[columna].unique()}\n")
    print(f"Las frecuencias de los valores únicos de las categorías son: {df2_cat[columna].value_counts()} ")

Las columnas del DataFrame de variables categóricas son Index(['title', 'Production company', 'Release date', 'Running time',
       'Country', 'Language', 'Release date (datetime)', 'rotten_tomatoes',
       'Directed by', 'Produced by', 'Written by', 'Based on', 'Starring',
       'Music by', 'Distributed by', 'Budget', 'Box office', 'Story by',
       'Narrated by', 'Cinematography', 'Edited by', 'Screenplay by',
       'Production companies', 'Adaptation by', 'Traditional', 'Simplified'],
      dtype='object')
 
----------- ESTAMOS ANALIZANDO LA COLUMNA: 'TITLE' -----------

Sus valores únicos son: ['Academy Award Review of' 'Snow White and the Seven Dwarfs' 'Pinocchio'
 'Fantasia' 'The Reluctant Dragon' 'Dumbo' 'Bambi' 'Saludos Amigos'
 'Victory Through Air Power' 'The Three Caballeros' 'Make Mine Music'
 'Song of the South' 'Fun and Fancy Free' 'Melody Time'
 'So Dear to My Heart' 'The Adventures of Ichabod and Mr. Toad'
 'Cinderella' 'Treasure Island' 'Alice in Wonderland'
 'The

Sus valores únicos son: ["['May 19, 1937']"
 "['December 21, 1937 ( Carthay Circle Theatre , Los Angeles , CA )', 'February 4, 1938 (United States)']"
 "['February 7, 1940 ( Center Theatre )', 'February 23, 1940 (United States)']"
 "['November 13, 1940']" "['June 20, 1941']"
 "['October 23, 1941 (New York City)', 'October 31, 1941 (U.S.)']"
 "['August 9, 1942 (World Premiere-London)', 'August 13, 1942 (Premiere-New York City)', 'August 21, 1942 (U.S.)']"
 "['August 24, 1942 (World Premiere-Rio de Janeiro)', 'February 6, 1943 (U.S. Premiere-Boston)', 'February 19, 1943 (U.S.)']"
 "['July 17, 1943']"
 "['December 21, 1944 (Mexico City)', 'February 3, 1945 (US)']"
 "['April 20, 1946 (Premiere-New York City)', 'August 15, 1946 (U.S.)']"
 "['November 12, 1946 (Premiere: Atlanta, Georgia)', 'November 20, 1946']"
 "['September 27, 1947']" 'May 27, 1948'
 "['November 29, 1948 (Chicago, Illinois)', 'January 19, 1949 (Indianapolis, Indiana)']"
 "['October 5, 1949']"
 "['February 15, 1950 (Boston

### LIMPIEZA

In [12]:
#Borramos columnas innecesarias para el análsis
columnas_a_eliminar = ['Release date','Running time', 'metascore', 'rotten_tomatoes', 'Written by', 'Starring', 'Music by', 'Budget', 'Box office', 'Story by', 'Narrated by', 'Cinematography', 'Edited by', 'Screenplay by', 'Production companies', 'Adaptation by', 'Traditional', 'Simplified']
def eliminar_columnas(df, lista):
    df.drop( columns = lista, inplace= True)
eliminar_columnas(df,columnas_a_eliminar) 



In [13]:
df.shape

(432, 13)

In [14]:
df.columns


Index(['title', 'Production company', 'Country', 'Language',
       'Running time (int)', 'Budget (float)', 'Box office (float)',
       'Release date (datetime)', 'imdb', 'Directed by', 'Produced by',
       'Based on', 'Distributed by'],
      dtype='object')

In [15]:
#hacemos una variable de nuevas columnas que hará que las columnas aparezcan en minuscula y que remplace el espacio por comas
new_columns = {column : column.lower().replace(' ', '_') for column in df}

#renombramos las columnas antiguas por las nuevas
df.rename(columns = new_columns, inplace= True)

# comprobamos que se ha realizado
df.sample(10)

,title,production_company,country,language,running_time_(int),budget_(float),box_office_(float),release_date_(datetime),imdb,directed_by,produced_by,based_on,distributed_by
96,The Boatniks,Walt Disney Productions,United States,English,100.0,NaN,18607492.0,1970-07-01,5.5,Norman Tokar,Ron Miller,NaN,Buena Vista Distribution
264,Freaky Friday,"['Walt Disney Pictures', 'Gunn Films']",United States,English,97.0,26000000.0,160800000.0,2003-08-04,6.2,Mark Waters,Andrew Gunn,"['Freaky Friday', 'by', 'Mary Rodgers']",Buena Vista Pictures
41,The Shaggy Dog,Walt Disney Productions,United States,English,104.0,1000000.0,12300000.0,1959-03-19,4.4,Charles Barton,"['Walt Disney', 'Bill Walsh']","['The Hound of Florence', 'by', 'Felix Salten']",Buena Vista Distribution
49,Swiss Family Robinson,Walt Disney Productions,United States,English,126.0,4000000.0,40000000.0,1960-12-21,7.2,"['John McKimson', 'Ken Annakin']","['Bill Anderson', 'Basil Keys', 'Walt Disney']","['The Swiss Family Robinson', 'by', 'Johann Da...",Buena Vista Distribution
76,That Darn Cat!,Walt Disney Productions,United States,English,116.0,NaN,28068222.0,1965-12-02,6.8,Robert Stevenson,"['Walt Disney', 'Bill Walsh']",NaN,Buena Vista Distribution
85,"Charlie, the Lonesome Cougar",Walt Disney Productions,United States,English,75.0,NaN,NaN,1967-10-18,6.9,"['Rex Allen', 'Winston Hibler']","['Walt Disney', 'Winston Hibler', 'Charles L. ...",NaN,Buena Vista Distribution
142,The Omega Connection,Walt Disney Productions,United States,English,100.0,NaN,NaN,1979-12-21,NaN,Robert Clouse,Jan Williams,NaN,Buena Vista Distribution
427,Soul,"['Walt Disney Pictures', 'Pixar Animation Stud...",United States,English,NaN,150000000.0,NaN,2020-10-15,7.0,Pete Docter,Dana Murray,NaN,"['Walt Disney Studios', 'Motion Pictures']"
116,The Island at the Top of the World,Walt Disney Productions,United States,"['English', 'Swedish', 'Norwegian', 'Danish', ...",93.0,8000000.0,10000000.0,1974-12-20,6.3,Robert Stevenson,Winston Hibler,NaN,Buena Vista Distribution
131,The Rescuers,Walt Disney Productions,United States,English,77.0,7500000.0,169000000.0,1977-06-22,6.9,"['Wolfgang Reitherman', 'John Lounsbery', 'Art...","['Wolfgang Reitherman', 'Ron W. Miller']","['The Rescuers', 'and', 'Miss Bianca', 'by', '...",Buena Vista Distribution


In [16]:
# quitamos int, float y datatime del titulo
df.columns = [col.replace('_(int)', '') for col in df.columns]
df.columns = [col.replace('_(float)', '') for col in df.columns]
df.columns = [col.replace('_(datetime)', '') for col in df.columns]


In [17]:
df.head(1)

,title,production_company,country,language,running_time,budget,box_office,release_date,imdb,directed_by,produced_by,based_on,distributed_by
0,Academy Award Review of,Walt Disney Productions,United States,English,41.0,NaN,NaN,1937-05-19,7.2,NaN,NaN,NaN,NaN


In [18]:
df.columns

Index(['title', 'production_company', 'country', 'language', 'running_time',
       'budget', 'box_office', 'release_date', 'imdb', 'directed_by',
       'produced_by', 'based_on', 'distributed_by'],
      dtype='object')

 0   title                    432 non-null    object  OK  ***SI***

 1   Production company       392 non-null    object  Lista (ver como hacer) (ver como hacer) Hacer explode. USA --- Nulos ***SI***

 2   Release date             431 non-null    object  Tenemos lista hay otra columna con este dato Podemos hacer una nueva columna con el lugar de estreno dividido entre parentesis ***SI*** LUGAR DE ESTENO ()
 
 4   Country                  428 non-null    object  Lista (ver como hacer) Hacer explode ***SI***

 5   Language                 430 non-null    object  Lista (ver como hacer) Hacer explode ***SI***

 6   Running time (int)       422 non-null    float64 OK coger media para nulos ***SI***

 7   Budget (float)           273 non-null    float64 Ok gestionar nulos ***SI***

 8   Box office (float)       355 non-null    float64 ok gestionar nulos ***SI*** (GANADO EN TAQUILLA)

 9   Release date (datetime)  428 non-null    object  ok los nulos como una categoria nueva ***SI*** 

 10  imdb                     416 non-null    float64 ok  media mediana para nulos ***SI*** 

13  Directed by              431 non-null    object  tiene listas, quededarnos con el primero pero quitar las categorias que no vamos a usar ***SI*** 

14  Produced by              422 non-null    object nos quedamos con el primero ***SI*** 
  
16  Based on                 211 non-null    object lista convertir string, limpieza de caracteres ***SI*** 

19  Distributed by           429 non-null    object es una lista, nos quedamos con el primero ***SI***


 0   title                    432 non-null    object  OK  ***SI***

 1   Production company       392 non-null    object  Lista (ver como hacer) (ver como hacer) Hacer explode. USA --- Nulos ***SI***

 2   Release date             431 non-null    object  Tenemos lista hay otra columna con este dato Podemos hacer una nueva columna con el lugar de estreno dividido entre parentesis ***SI*** LUGAR DE ESTENO ()
 
 4   Country                  428 non-null    object  Lista (ver como hacer) Hacer explode ***SI***

 5   Language                 430 non-null    object  Lista (ver como hacer) Hacer explode ***SI***

 6   Running time (int)       422 non-null    float64 OK coger media para nulos ***SI***

 7   Budget (float)           273 non-null    float64 Ok gestionar nulos ***SI***

 8   Box office (float)       355 non-null    float64 ok gestionar nulos ***SI*** (GANADO EN TAQUILLA)

 9   Release date (datetime)  428 non-null    object  ok los nulos como una categoria nueva ***SI*** 

 10  imdb                     416 non-null    float64 ok  media mediana para nulos ***SI*** 

13  Directed by              431 non-null    object  tiene listas, quededarnos con el primero pero quitar las categorias que no vamos a usar ***SI*** 

14  Produced by              422 non-null    object nos quedamos con el primero ***SI*** 
  
16  Based on                 211 non-null    object lista convertir string, limpieza de caracteres ***SI*** 

19  Distributed by           429 non-null    object es una lista, nos quedamos con el primero ***SI***


Index(['title', 'production_company', 'country', 'language', 'running_time',
       'budget', 'box_office', 'release_date', 'imdb', 'directed_by',
       'produced_by', 'based_on', 'distributed_by'],
      dtype='object')

In [19]:
df.columns

Index(['title', 'production_company', 'country', 'language', 'running_time',
       'budget', 'box_office', 'release_date', 'imdb', 'directed_by',
       'produced_by', 'based_on', 'distributed_by'],
      dtype='object')

In [20]:
# Lista de columnas a transformar con explode
columns_to_explode = ['production_company', 'country', 'language']

# Iterar sobre cada columna y aplicar las transformaciones
for column in columns_to_explode:
    if column in df.columns:
        # Convertir todos los valores a cadenas para evitar errores
        df[column] = df[column].astype(str)
        # Quitar corchetes, eliminar comillas y dividir por comas
        df[column] = df[column].str.strip("[]").str.replace("'", "").str.split(", ")
        # Aplicar explode
        df = df.explode(column)
        # Limpiar espacios en blanco alrededor de los elementos
        df[column] = df[column].str.strip()

# Revisar la forma del DataFrame final
print(f"Forma después de aplicar explode a todas las columnas: {df.shape}")

# Mostrar las primeras filas del DataFrame final usando display
display(df.head())


Forma después de aplicar explode a todas las columnas: (974, 13)


,title,production_company,country,language,running_time,budget,box_office,release_date,imdb,directed_by,produced_by,based_on,distributed_by
0,Academy Award Review of,Walt Disney Productions,United States,English,41.0,NaN,NaN,1937-05-19,7.2,NaN,NaN,NaN,NaN
1,Snow White and the Seven Dwarfs,Walt Disney Productions,United States,English,83.0,1490000.0,418000000.0,1937-12-21,7.6,"['David Hand (supervising)', 'William Cottrell...",Walt Disney,"['Snow White', 'by The', 'Brothers Grimm']",RKO Radio Pictures
2,Pinocchio,Walt Disney Productions,United States,English,88.0,2600000.0,164000000.0,1940-02-07,7.4,"['Ben Sharpsteen', 'Hamilton Luske', 'Bill Rob...",Walt Disney,"['The Adventures of Pinocchio', 'by', 'Carlo C...",RKO Radio Pictures
3,Fantasia,Walt Disney Productions,United States,English,126.0,2280000.0,83300000.0,1940-11-13,7.8,"['Samuel Armstrong', 'James Algar', 'Bill Robe...","['Walt Disney', 'Ben Sharpsteen']",NaN,"['Walt Disney Productions', 'RKO Radio Pictures']"
4,The Reluctant Dragon,Walt Disney Productions,United States,English,74.0,600000.0,960000.0,1941-06-20,6.9,"['Alfred Werker', '(live action)', 'Hamilton L...",Walt Disney,NaN,RKO Radio Pictures


### NULOS

In [21]:
#Exploracion columnas 

def eda_columnas(df):
    
    # Separar columnas categóricas y numéricas
    columnas_categoricas = df.select_dtypes(include=['object', 'category']).columns
    columnas_numericas = df.select_dtypes(include=['number']).columns
    
    display("Columnas categóricas:", columnas_categoricas)
    display("Columnas numéricas:", columnas_numericas)
    
    # Exploración de columnas categóricas
    for columna in columnas_categoricas:
        display(f"Exploración de la columna categórica: {columna}")
        datos_columna = df[columna]  # Extrae los datos de la columna específica

        # Descripción estadística
        display("Descripción estadística:")
        display(datos_columna.describe(include='all'))
        
        # Valores nulos
        display("Conteo de valores nulos:")
        display(datos_columna.isnull().sum())
        
        # Valores duplicados
        display("Número de filas duplicadas:")
        display(df.duplicated(subset=[columna]).sum())
       
        # Valores únicos
        display("Conteo de valores únicos:")
        display(datos_columna.nunique())
        
        # Conteo de cada valor
        display("Conteo de valores:")
        display(datos_columna.value_counts())
        
        # Valor más frecuente (moda)
        if not datos_columna.mode().empty:
            display("Valor más frecuente (moda):")
            display(datos_columna.mode().iloc[0])
        
        display("-" * 30)

    # Exploración de columnas numéricas
    for columna in columnas_numericas:
        display(f"Exploración de la columna numérica: {columna}")
        datos_columna = df[columna]  # Extrae los datos de la columna específica

        # Descripción estadística
        display("Descripción estadística:")
        display(datos_columna.describe())
        
        # Valores nulos
        display("Conteo de valores nulos:")
        display(datos_columna.isnull().sum())
        
        # Valores duplicados
        display("Número de filas duplicadas:")
        display(df.duplicated(subset=[columna]).sum())
       
        # Valores únicos
        display("Conteo de valores únicos:")
        display(datos_columna.nunique())
        
        display("-" * 30)

# Llamada a la función con tu DataFrame modificado llamado 'df'
eda_columnas(df)


'Columnas categóricas:'

Index(['title', 'production_company', 'country', 'language', 'release_date',
       'directed_by', 'produced_by', 'based_on', 'distributed_by'],
      dtype='object')

'Columnas numéricas:'

Index(['running_time', 'budget', 'box_office', 'imdb'], dtype='object')

'Exploración de la columna categórica: title'

...

'Número de filas duplicadas:'

np.int64(924)

'Conteo de valores únicos:'

49

'------------------------------'

In [22]:
df.isnull().sum()

title                   0
production_company      0
country                 0
language                0
running_time           27
budget                252
box_office            140
release_date           24
imdb                   39
directed_by             1
produced_by            21
based_on              560
distributed_by          9
dtype: int64

In [23]:
# Porcentaje de valores nulos
print("Porcentaje de valores nulos por columna:")
display((df.isnull().sum() / len(df) * 100).round(2))
print("----"*10)

Porcentaje de valores nulos por columna:


title                  0.00
production_company     0.00
country                0.00
language               0.00
running_time           2.77
budget                25.87
box_office            14.37
release_date           2.46
imdb                   4.00
directed_by            0.10
produced_by            2.16
based_on              57.49
distributed_by         0.92
dtype: float64

----------------------------------------


In [24]:
df.head(1)

,title,production_company,country,language,running_time,budget,box_office,release_date,imdb,directed_by,produced_by,based_on,distributed_by
0,Academy Award Review of,Walt Disney Productions,United States,English,41.0,NaN,NaN,1937-05-19,7.2,NaN,NaN,NaN,NaN


In [25]:
#Gestion nulos variables categoricas
def variables_categoricas(df):
   
    # Identificar columnas categóricas con valores nulos
    nulos_categoricas = df[df.columns[df.isnull().any()]].select_dtypes(include="O").columns
    print("Las columnas categóricas que tienen nulos son : \n")
    print(nulos_categoricas)

   # Asegurar que la columna 'release_date' esté en formato datetime
    df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

    # Reemplazar los valores nulos en 'release_date' con la fecha estándar '1900-01-01'
    df['release_date'].fillna(pd.to_datetime('1900-01-01'), inplace=True)

    # Verificar la presencia de la fecha '1900-01-01'
    value_counts = df['release_date'].value_counts()
    display("Conteo de fechas en 'release_date':")
    display(value_counts)

    # Verificar que no queden valores nulos
    nulos_restantes = df['release_date'].isnull().sum()
    display(f"Valores nulos restantes en release_date: {nulos_restantes}")

    # Visualizar las primeras filas de la columna para ver los cambios
    display("Primeras filas de 'release_date':")
    display(df['release_date'].head(10))

    # Mostrar una muestra aleatoria de la columna para verificar
    display("Muestra aleatoria de 'release_date':")
    display(df['release_date'].sample(10))

#Llamada a la funcion
variables_categoricas(df)


Las columnas categóricas que tienen nulos son : 

Index(['release_date', 'directed_by', 'produced_by', 'based_on',
       'distributed_by'],
      dtype='object')


"Conteo de fechas en 'release_date':"

release_date
2016-08-12    36
1900-01-01    24
2011-04-22    20
2007-10-10    16
2004-06-13    16
              ..
2018-11-05     1
2019-06-11     1
1940-02-07     1
1937-12-21     1
1937-05-19     1
Name: count, Length: 422, dtype: int64

'Valores nulos restantes en release_date: 0'

"Primeras filas de 'release_date':"

0   1937-05-19
1   1937-12-21
2   1940-02-07
3   1940-11-13
4   1941-06-20
5   1941-10-23
6   1942-08-09
7   1942-08-24
7   1942-08-24
7   1942-08-24
Name: release_date, dtype: datetime64[ns]

"Muestra aleatoria de 'release_date':"

174   1990-10-03
363   2013-10-20
319   2007-10-10
224   1998-03-27
299   2007-03-23
59    1962-09-26
195   1994-10-28
116   1974-12-20
142   1979-12-21
334   2008-10-26
Name: release_date, dtype: datetime64[ns]

In [26]:
def variables_numericas(dg):
   
    # Identificar columnas numéricas con valores nulos
    nulos_numericas = df[df.columns[df.isnull().any()]].select_dtypes(include=np.number).columns
    print("Las columnas numéricas que tienen nulos son : \n")
    print(nulos_numericas)

    # Mostrar el porcentaje de valores nulos 
    print(df[nulos_numericas].isnull().sum() / df.shape[0] * 100)
   
    # Reemplazar nulos con la mediana
    columnas_mediana =['running_time','imdb']
    mediana = df[columnas_mediana].median()
    df[columnas_mediana] = df[columnas_mediana].fillna(mediana)
    
    # Verificar que los nulos en las columnas de mediana fueron reemplazados
    print("\nDespués de reemplazar nulos con la mediana:")
    print(df[columnas_mediana].isnull().sum())

    # Reemplazar nulos con fillna 0
    columnas_fillna_0=['budget', 'box_office']
    df[columnas_fillna_0] = df[columnas_fillna_0].fillna(0)

   # Verificar que los nulos en las columnas de fillna(0) fueron reemplazados
    print("\nDespués de reemplazar nulos con 0:")
    print(df[columnas_fillna_0].isnull().sum())

    # Copiar el DataFrame 
    df_copia = df.copy()
    
    # Guardar el DataFrame en un archivo CSV
    df_copia.to_csv("DisneyMoviesDataset_nonulls.csv", index=False)

    return df_copia

# Imputar valores nulos en variables numéricas
df_final = variables_numericas(df)
df_final.isnull().sum()

Las columnas numéricas que tienen nulos son : 

Index(['running_time', 'budget', 'box_office', 'imdb'], dtype='object')
running_time     2.772074
budget          25.872690
box_office      14.373717
imdb             4.004107
dtype: float64

Después de reemplazar nulos con la mediana:
running_time    0
imdb            0
dtype: int64

Después de reemplazar nulos con 0:
budget        0
box_office    0
dtype: int64


title                   0
production_company      0
country                 0
language                0
running_time            0
budget                  0
box_office              0
release_date            0
imdb                    0
directed_by             1
produced_by            21
based_on              560
distributed_by          9
dtype: int64

In [27]:
df_final.head()

,title,production_company,country,language,running_time,budget,box_office,release_date,imdb,directed_by,produced_by,based_on,distributed_by
0,Academy Award Review of,Walt Disney Productions,United States,English,41.0,0.0,0.0,1937-05-19,7.2,NaN,NaN,NaN,NaN
1,Snow White and the Seven Dwarfs,Walt Disney Productions,United States,English,83.0,1490000.0,418000000.0,1937-12-21,7.6,"['David Hand (supervising)', 'William Cottrell...",Walt Disney,"['Snow White', 'by The', 'Brothers Grimm']",RKO Radio Pictures
2,Pinocchio,Walt Disney Productions,United States,English,88.0,2600000.0,164000000.0,1940-02-07,7.4,"['Ben Sharpsteen', 'Hamilton Luske', 'Bill Rob...",Walt Disney,"['The Adventures of Pinocchio', 'by', 'Carlo C...",RKO Radio Pictures
3,Fantasia,Walt Disney Productions,United States,English,126.0,2280000.0,83300000.0,1940-11-13,7.8,"['Samuel Armstrong', 'James Algar', 'Bill Robe...","['Walt Disney', 'Ben Sharpsteen']",NaN,"['Walt Disney Productions', 'RKO Radio Pictures']"
4,The Reluctant Dragon,Walt Disney Productions,United States,English,74.0,600000.0,960000.0,1941-06-20,6.9,"['Alfred Werker', '(live action)', 'Hamilton L...",Walt Disney,NaN,RKO Radio Pictures
